In [ ]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=fed9588acb0486f4ebdfd53a5c51b3cd138c5e99ffee2c3c67cb5564f9d407b0
  Stored in directory: /root/.cache/pip/wheels/62/f9/aa/687bd54342d2981bc78e22ee9b9bc39f92006e344e7aa1e0ac
Successfully built h2o


In [ ]:
## Import Libraries

import h2o
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from h2o.automl import H2OAutoML

In [ ]:
h2o.init()     ## Initiate h2o server

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.26" 2025-01-21; OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpjygwty8r
  JVM stdout: /tmp/tmpjygwty8r/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpjygwty8r/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 months and 4 days
H2O_cluster_name:,H2O_from_python_unknownUser_nqqr2r
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
## Reading The Data
iris=load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['target']=iris.target

In [ ]:
##Specifying The target & features
X=iris.feature_names
y='target'

In [11]:
df['target']=df['target'].astype(str)    ## Converting target column to string as prefered to H2O


In [12]:
## Spliting The dataset
train_df, test_df=train_test_split(df,test_size=0.2,random_state=42)

In [13]:
## Converting pandas df to H2O dataframe
train_h2o=h2o.H2OFrame(train_df)
test_h2o=h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
# Convert target column to categorical
train_h2o[y]=train_h2o[y].asfactor()
test_h2o[y]=test_h2o[y].asfactor()

In [15]:
## Building $ Training AutoML

aml=H2OAutoML(max_models=10,max_runtime_secs=360,seed=42)   ## Training only 10 models or during 6 min total trainig time
aml.train(x=X,y=y,training_frame=train_h2o)

AutoML progress: |████
05:34:59.715: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 120.0.

███████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_1_20250306_53447


GLM Model: summary
    family       link         regularization               lambda_search                                                                    number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------  -------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------------------------------------
    multinomial  multinomial  Ridge ( lambda = 2.912E-4 )  nlambda = 30, lambda.max = 43.307, lambda.min = 2.912E-4, lambda.1se = 0.001216  15                            12                             100                     AutoML_1_20250306_53447_training_py_1_sid_bbe3

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.015864151889199693
RMSE: 0.12595297491206667
LogLoss: 0.055842850234971826
Null degrees of freedom: 119
Residual degrees of freedom: 105
Null deviance: 263.61694407071036
Residual deviance: 13.402284056393238
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1    2    Error      Rate
---  ---  ---  ---------  -------
40   0    0    0          0 / 40
0    40   1    0.0243902  1 / 41
0    1    38   0.025641   1 / 39
40   41   39   0.0166667  2 / 120

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.983333
2    1
3    1

ModelMetricsMultinomialGLM: glm
** Reported on cross-validation data. **

MSE: 0.027452710000474324
RMSE: 0.16568859345312315
LogLoss: 0.0861235880817368
Null degrees of freedom: 119
Residual degrees of freedom: 105
Null deviance: 268.2758084968263
Residual deviance: 20.66966113961683
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1    2    Error      Rate
---  ---  ---  ---------  -------
40   0    0    0          0 / 40
0    37   4    0.097561   4 / 41
0    1    38   0.025641   1 / 39
40   38   42   0.0416667  5 / 120

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.958333
2    1
3    1

Cross-Validation Metrics Summary: 
                         mean       sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  ---------  ---------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.958333   0.0294628  1             0.958333      0.958333      0.958333      0.916667
aic                      nan        0          nan           nan           nan           nan           nan
auc                      nan        0          nan           nan           nan           nan           nan
err                      0.0416667  0.0294628  0             0.0416667     0.0416667     0.0416667     0.0833333
err_count                1          0.707107   0             1             1             1             2
loglikelihood            0          0          0             0             0             0             0
logloss                  0.0796585  0.048228   0.00944584    0.126556      0.109892      0.0513342     0.101065
max_per_class_error      0.115325   0.0751364  0             0.0909091     0.142857      0.142857   

In [16]:
## Dispalying Ranked models list with their metrics
lb=aml.leaderboard
print(lb)

model_id                                                  mean_per_class_error    logloss      rmse        mse
GLM_1_AutoML_1_20250306_53447                                        0.0410673  0.0861236  0.165689  0.0274527
StackedEnsemble_AllModels_1_AutoML_1_20250306_53447                  0.0496143  0.121962   0.1908    0.0364048
StackedEnsemble_BestOfFamily_1_AutoML_1_20250306_53447               0.0581614  0.116957   0.187078  0.0349983
XGBoost_3_AutoML_1_20250306_53447                                    0.0585783  0.202605   0.229621  0.0527259
GBM_2_AutoML_1_20250306_53447                                        0.0585783  0.182925   0.224998  0.0506242
GBM_5_AutoML_1_20250306_53447                                        0.0671253  0.209447   0.241592  0.0583669
DRF_1_AutoML_1_20250306_53447                                        0.0671253  0.185999   0.223757  0.0500671
GBM_4_AutoML_1_20250306_53447                                        0.0671253  0.184961   0.230702  0.0532234
X

In [17]:
## Predicting The Test Data
preds=aml.leader.predict(test_h2o)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [18]:
## Showing Confusion Metrics on test data
perf=aml.leader.model_performance(test_h2o)
print(perf.confusion_matrix())

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1    2    Error    Rate
---  ---  ---  -------  ------
10   0    0    0        0 / 10
0    9    0    0        0 / 9
0    0    11   0        0 / 11
10   9    11   0        0 / 30
